# References

* http://www.sciencedirect.com/science/article/pii/S0925231203003722
* http://ac.els-cdn.com/S0957417400000270/1-s2.0-S0957417400000270-main.pdf?_tid=3a06fc62-1d5b-11e6-877f-00000aab0f27&acdnat=1463619013_cba9f7ee840313639128ce15571f73ac
* Technical Analysis of Stock Trends, Robert D. Edwards and John Magee
* https://www.jbs.cam.ac.uk/fileadmin/user_upload/research/workingpapers/wp0030.pdf
* http://www.sciencedirect.com/science/article/pii/0261560692900483

In [1]:
%matplotlib inline

%load_ext autoreload
%autoreload 2

%load_ext version_information
%version_information deap, matplotlib, numpy, pandas, seaborn, sklearn

Software versions
Python 3.5.1 64bit [GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]
IPython 4.1.2
OS Linux 3.13.0 86 generic x86_64 with debian jessie sid
deap 1.0
matplotlib 1.5.1
numpy 1.10.4
pandas 0.18.0
seaborn 0.7.0
sklearn 0.17.1
Thu May 19 04:18:45 2016 BST

In [48]:
%matplotlib inline

import ast
from IPython.display import display
from ipywidgets import widgets
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from collections import OrderedDict as od
import random
import re
import seaborn as sns

#pd.set_option('html', False)
np.set_printoptions(threshold=np.nan)
sns.set()

# Technical Indicators

In [185]:
def stoch_K(close, window):
    low = close.rolling(window, center = False).min()
    high = close.rolling(window, center = False).max()
    
    return 100 * (close - low) / (high - low)

def stoch_D(K, window):
    return K.rolling(window, center = False).mean()

def slow_D(D, window):
    return D.rolling(window, center = False).mean()

def momentum(close, window):
    dif = lambda x: x[-1] - x[0]
    
    return close.rolling(window, center = False).apply(dif)

def roc(close, window):
    ratio = lambda x: x[-1] / x[0]
    
    return 100 * close.rolling(window, center = False).apply(ratio)

def lw_R(close, window):
    low = close.rolling(window, center = False).min()
    high = close.rolling(window, center = False).max()
    
    return 100 * (high - close) / (high - low)
    
def ad_osc(close, window):
    low = close.rolling(window, center = False).min()
    high = close.rolling(window, center = False).max()
    prev_close = close.rolling(window, center = False).apply(lambda x: x[-2])
    
    return (high - prev_close) / (high - low)

def disp(close, window):
    MA = close.rolling(window, center = False).mean()
    
    return 100 * close / MA

def oscp(close, window1, window2):
    MA1 = close.rolling(window1, center = False).mean()
    MA2 = close.rolling(window2, center = False).mean()
    
    return (MA1 - MA2) / MA1

def rsi(close, window):
    up, down = close.diff().copy(), close.diff().copy()
    up[up < 0] = 0
    down[down > 0] = 0
    
    RS = up.rolling(window, center = False).mean() / down.rolling(window, center = False).mean().abs()
    
    return 100 - (100 / (1 + RS))

# Feature Engineering of Feature Set 1 - turned out to be meaningless

In [158]:
lob_data = pd.read_csv('../btc-data/BTC_LOB_collected.csv')

In [159]:
lob_features10 = pd.DataFrame(lob_data)
lob_features10.set_index(lob_data['Unnamed: 0'], inplace = True)
lob_features10.drop('Unnamed: 0', axis = 1, inplace = True)
lob_features10.index = pd.to_datetime(lob_features10.index)
lob_features10['asks'] = lob_features10['asks'].map(ast.literal_eval)
lob_features10['bids'] = lob_features10['bids'].map(ast.literal_eval)

In [160]:
lob_features10['total ask volume'] = lob_features10['asks'].map(lambda x: sum(x.values()))
lob_features10['total bid volume'] = lob_features10['bids'].map(lambda x: sum(x.values()))
lob_features10['ask price'] = lob_features10['asks'].map(min)
lob_features10['bid price'] = lob_features10['bids'].map(max)

In [161]:
lob_features10['bid-ask spread'] = lob_features10['ask price'] - lob_features10['bid price']
lob_features10['mid price'] = (lob_features10['ask price'] + lob_features10['bid price'])/2
lob_features10['ask price spread'] = lob_features10['asks'].map(max) - lob_features10['ask price']
lob_features10['bid price spread'] = lob_features10['bid price'] - lob_features10['bids'].map(min)
lob_features10['mean ask volume'] = lob_features10['total ask volume'] / 20
lob_features10['mean bid volume'] = lob_features10['total bid volume'] / 20
lob_features10['mean ask price'] = lob_features10['asks'].map(sum) / 20
lob_features10['mean bid price'] = lob_features10['bids'].map(sum) / 20

In [162]:
lob_features10

,asks,bids,total ask volume,total bid volume,ask price,bid price,bid-ask spread,mid price,ask price spread,bid price spread,mean ask volume,mean bid volume,mean ask price,mean bid price
Unnamed: 0,,,,,,,,,,,,,,
2016-01-01 00:00:00,"{432.84: 39.75, 432.36: 6.9631, 432.83: 9.2466...","{429.31: 3.88, 429.33: 39.75, 429.47: 32.0333,...",318.632799,387.595528,430.89,429.84,1.05,430.365,2.11,1.52,15.931640,19.379776,432.1405,429.0090
2016-01-01 00:00:10,"{432.88: 41.73, 432.48: 13.8492, 432.99: 9.257...","{429.38: 16.114, 429.2: 48.9702, 429.33: 39.75...",318.629099,323.428038,430.89,429.84,1.05,430.365,2.11,1.19,15.931455,16.171402,432.1875,429.1115
2016-01-01 00:00:20,"{432.64: 6.9491, 432.52: 1.17, 432.84: 39.75, ...","{429.41: 9.2521, 429.33: 39.75, 429.01: 14.836...",290.919269,354.135003,430.89,429.86,1.03,430.375,2.44,1.54,14.545963,17.706750,432.2465,429.0100
2016-01-01 00:00:30,"{432.11: 4.635, 432.36: 6.9556000000000004, 43...","{428.65: 0.02799486, 429.33: 39.75, 429.35: 16...",232.153855,172.031328,430.89,429.89,1.00,430.390,2.80,1.56,11.607693,8.601566,432.3350,429.0750
2016-01-01 00:00:40,"{432.11: 4.635, 432.36: 6.9556000000000004, 43...","{429.49: 32.0333, 428.65: 0.02799486, 428.83: ...",290.952369,322.855038,430.89,429.90,0.99,430.395,2.44,1.25,14.547618,16.142752,432.2360,429.1115
2016-01-01 00:00:50,"{432.11: 4.635, 432.36: 6.9556000000000004, 43...","{428.65: 0.02799486, 429.45: 39.75, 429.28: 0....",290.952369,287.730638,430.89,429.90,0.99,430.395,2.44,1.25,14.547618,14.386532,432.2360,429.1010
2016-01-01 00:01:00,"{432.11: 4.635, 432.36: 6.9556000000000004, 43...","{428.65: 0.02799486, 429.2: 48.9764, 429.01: 1...",290.871738,374.632128,430.89,429.90,0.99,430.395,2.44,1.58,14.543587,18.731606,432.2360,428.9660
2016-01-01 00:01:10,"{432.11: 4.635, 432.35: 6.9594000000000005, 43...","{429.45: 39.75, 429.47: 14.916, 428.65: 0.0279...",290.875538,322.244428,430.89,429.90,0.99,430.395,2.44,1.55,14.543777,16.112221,432.2355,429.0825
2016-01-01 00:01:20,"{432.11: 4.635, 432.35: 6.8916, 433.0: 0.1, 43...","{429.02: 29.13, 429.91: 32.0333, 429.47: 2.35,...",279.887738,309.666428,430.89,429.91,0.98,430.400,2.32,1.56,13.994387,15.483321,432.2295,429.1055


In [163]:
lob_features10.drop(['asks', 'bids'], axis = 1, inplace=True)
lob_features10.to_csv(path_or_buf='../btc-data/BTC_LOB_simple_10s.csv')

In [164]:
lob_features30 = lob_features10.reindex(pd.date_range(start = lob_features10.index[0],
                                                      end = lob_features10.index[-1], freq='30s'))

lob_features60 = lob_features10.reindex(pd.date_range(start = lob_features10.index[0],
                                                      end = lob_features10.index[-1], freq='60s'))

lob_features300 = lob_features10.reindex(pd.date_range(start = lob_features10.index[0],
                                                       end = lob_features10.index[-1], freq='300s'))

lob_features600 = lob_features10.reindex(pd.date_range(start = lob_features10.index[0],
                                                       end = lob_features10.index[-1], freq='600s'))

In [171]:
lob_features30

,total ask volume,total bid volume,ask price,bid price,bid-ask spread,mid price,ask price spread,bid price spread,mean ask volume,mean bid volume,mean ask price,mean bid price
2016-01-01 00:00:00,318.632799,387.595528,430.89,429.84,1.05,430.365,2.11,1.52,15.931640,19.379776,432.1405,429.0090
2016-01-01 00:00:30,232.153855,172.031328,430.89,429.89,1.00,430.390,2.80,1.56,11.607693,8.601566,432.3350,429.0750
2016-01-01 00:01:00,290.871738,374.632128,430.89,429.90,0.99,430.395,2.44,1.58,14.543587,18.731606,432.2360,428.9660
2016-01-01 00:01:30,285.901138,307.380028,430.89,429.92,0.97,430.405,2.44,1.57,14.295057,15.369001,432.3270,429.0700
2016-01-01 00:02:00,312.929338,327.741478,430.89,429.95,0.94,430.420,2.28,1.30,15.646467,16.387074,432.3600,429.1935
2016-01-01 00:02:30,312.832581,326.718378,430.89,430.05,0.84,430.470,2.28,1.40,15.641629,16.335919,432.3450,429.2110
2016-01-01 00:03:00,279.716681,336.122423,430.89,430.09,0.80,430.490,2.28,1.41,13.985834,16.806121,432.3365,429.2315
2016-01-01 00:03:30,279.744681,306.098773,430.89,430.14,0.75,430.515,2.28,1.44,13.987234,15.304939,432.3665,429.3355
2016-01-01 00:04:00,279.871681,295.116008,430.89,430.17,0.72,430.530,2.33,1.52,13.993584,14.755800,432.3890,429.2450
2016-01-01 00:04:30,231.934879,179.662946,430.89,430.19,0.70,430.540,2.61,2.07,11.596744,8.983147,432.4435,428.9195


In [172]:
lob_features60

,total ask volume,total bid volume,ask price,bid price,bid-ask spread,mid price,ask price spread,bid price spread,mean ask volume,mean bid volume,mean ask price,mean bid price
2016-01-01 00:00:00,318.632799,387.595528,430.89,429.84,1.05,430.365,2.11,1.52,15.931640,19.379776,432.1405,429.0090
2016-01-01 00:01:00,290.871738,374.632128,430.89,429.90,0.99,430.395,2.44,1.58,14.543587,18.731606,432.2360,428.9660
2016-01-01 00:02:00,312.929338,327.741478,430.89,429.95,0.94,430.420,2.28,1.30,15.646467,16.387074,432.3600,429.1935
2016-01-01 00:03:00,279.716681,336.122423,430.89,430.09,0.80,430.490,2.28,1.41,13.985834,16.806121,432.3365,429.2315
2016-01-01 00:04:00,279.871681,295.116008,430.89,430.17,0.72,430.530,2.33,1.52,13.993584,14.755800,432.3890,429.2450
2016-01-01 00:05:00,217.439947,342.194493,431.06,430.19,0.87,430.625,2.74,1.50,10.871997,17.109725,432.6220,429.2145
2016-01-01 00:06:00,361.630784,294.113648,432.40,430.54,1.86,431.470,1.56,1.71,18.081539,14.705682,433.3085,429.6675
2016-01-01 00:07:00,347.402014,395.102693,432.52,431.46,1.06,431.990,1.45,2.11,17.370101,19.755135,433.3250,430.2590
2016-01-01 00:08:00,347.029411,338.648408,432.53,431.46,1.07,431.995,1.44,2.10,17.351471,16.932420,433.3880,430.4625
2016-01-01 00:09:00,414.224478,267.615308,432.53,431.46,1.07,431.995,1.43,2.45,20.711224,13.380765,433.3280,430.1980


In [173]:
lob_features300

,total ask volume,total bid volume,ask price,bid price,bid-ask spread,mid price,ask price spread,bid price spread,mean ask volume,mean bid volume,mean ask price,mean bid price
2016-01-01 00:00:00,318.632799,387.595528,430.89,429.84,1.05,430.365,2.11,1.52,15.931640,19.379776,432.1405,429.0090
2016-01-01 00:05:00,217.439947,342.194493,431.06,430.19,0.87,430.625,2.74,1.50,10.871997,17.109725,432.6220,429.2145
2016-01-01 00:10:00,414.543886,305.888470,432.41,431.46,0.95,431.935,1.49,2.53,20.727194,15.294424,433.1320,430.1110
2016-01-01 00:15:00,183.180919,222.309648,432.38,431.46,0.92,431.920,1.49,2.67,9.159046,11.115482,433.0830,430.0345
2016-01-01 00:20:00,311.297511,398.569889,432.42,431.46,0.96,431.940,1.47,2.45,15.564876,19.928494,433.2915,430.2975
2016-01-01 00:25:00,419.611941,434.477720,432.40,431.46,0.94,431.930,1.52,2.44,20.980597,21.723886,433.2520,430.1780
2016-01-01 00:30:00,234.429204,264.975279,432.39,431.46,0.93,431.925,1.41,2.45,11.721460,13.248764,433.0735,430.3140
2016-01-01 00:35:00,427.122203,306.567417,432.40,430.45,1.95,431.425,1.50,1.62,21.356110,15.328371,433.1205,429.7195
2016-01-01 00:40:00,248.873168,253.628860,431.11,430.09,1.02,430.600,2.35,1.41,12.443658,12.681443,432.6055,429.3755
2016-01-01 00:45:00,216.691350,225.566000,431.02,430.46,0.56,430.740,2.77,1.67,10.834567,11.278300,432.7370,429.4195


In [174]:
lob_features600

,total ask volume,total bid volume,ask price,bid price,bid-ask spread,mid price,ask price spread,bid price spread,mean ask volume,mean bid volume,mean ask price,mean bid price
2016-01-01 00:00:00,318.632799,387.595528,430.89,429.84,1.05,430.365,2.11,1.52,15.931640,19.379776,432.1405,429.0090
2016-01-01 00:10:00,414.543886,305.888470,432.41,431.46,0.95,431.935,1.49,2.53,20.727194,15.294424,433.1320,430.1110
2016-01-01 00:20:00,311.297511,398.569889,432.42,431.46,0.96,431.940,1.47,2.45,15.564876,19.928494,433.2915,430.2975
2016-01-01 00:30:00,234.429204,264.975279,432.39,431.46,0.93,431.925,1.41,2.45,11.721460,13.248764,433.0735,430.3140
2016-01-01 00:40:00,248.873168,253.628860,431.11,430.09,1.02,430.600,2.35,1.41,12.443658,12.681443,432.6055,429.3755
2016-01-01 00:50:00,254.093468,239.650073,431.05,430.16,0.89,430.605,2.64,1.81,12.704673,11.982504,432.7790,429.2280
2016-01-01 01:00:00,280.695908,189.337964,431.05,429.80,1.25,430.425,2.12,1.46,14.034795,9.466898,432.2610,429.0650
2016-01-01 01:10:00,254.614361,360.892833,431.35,429.43,1.92,430.390,2.45,1.11,12.730718,18.044642,432.5795,428.8460
2016-01-01 01:20:00,248.382690,304.781962,431.38,430.82,0.56,431.100,2.31,2.14,12.419135,15.239098,432.5520,429.2240
2016-01-01 01:30:00,257.805188,214.429454,431.23,429.80,1.43,430.515,2.57,1.50,12.890259,10.721473,432.4005,428.8950


In [175]:
lob_features30.to_csv(path_or_buf='../btc-data/BTC_LOB_simple_30s.csv')
lob_features60.to_csv(path_or_buf='../btc-data/BTC_LOB_simple_60s.csv')
lob_features300.to_csv(path_or_buf='../btc-data/BTC_LOB_simple_300s.csv')
lob_features600.to_csv(path_or_buf='../btc-data/BTC_LOB_simple_600s.csv')

# Feature Engineering of Feature Set 2 - better technical indicators

In [189]:
lob_techind10 = pd.DataFrame(lob_features10['mid price'].copy(), index = lob_features10.index)
lob_techind30 = pd.DataFrame(lob_features30['mid price'].copy(), index = lob_features30.index)
lob_techind60 = pd.DataFrame(lob_features60['mid price'].copy(), index = lob_features60.index)
lob_techind300 = pd.DataFrame(lob_features300['mid price'].copy(), index = lob_features300.index)
lob_techind600 = pd.DataFrame(lob_features600['mid price'].copy(), index = lob_features600.index)

In [190]:
def generate_features(frame, freq):
    close = frame['mid price']
    frame['K360'] = stoch_K(close, 360)
    frame['K180'] = stoch_K(close, 180)
    frame['K60'] = stoch_K(close, 60)
    frame['D360'] = stoch_D(frame['K360'], 360)
    frame['D180'] = stoch_D(frame['K180'], 180)
    frame['D60'] = stoch_D(frame['K60'], 60)
    frame['sD360'] = slow_D(frame['D360'], 360)
    frame['sD180'] = slow_D(frame['D180'], 180)
    frame['sD60'] = slow_D(frame['D60'], 60)
    frame['MOM360'] = momentum(close, 360)
    frame['MOM180'] = momentum(close, 180)
    frame['MOM60'] = momentum(close, 60)
    frame['ROC360'] = roc(close, 360)
    frame['ROC180'] = roc(close, 180)
    frame['ROC60'] = roc(close, 60)
    frame['LWR360'] = lw_R(close, 360)
    frame['LWR180'] = lw_R(close, 180)
    frame['LWR60'] = lw_R(close, 60)
    frame['ADOSC360'] = ad_osc(close, 360)
    frame['ADOSC180'] = ad_osc(close, 180)
    frame['ADOSC60'] = ad_osc(close, 60)
    frame['DISP360'] = disp(close, 360)
    frame['DISP180'] = disp(close, 180)
    frame['DISP60'] = disp(close, 60)
    frame['OSCP180-360'] = oscp(close, 180, 360)
    frame['OSCP60-180'] = oscp(close, 60, 180)
    frame['RSI360'] = rsi(close, 360)
    frame['RSI180'] = rsi(close, 180)
    frame['RSI60'] = rsi(close, 60)
    
    frame['mid price'] = frame['mid price'].shift(-1)
    frame.set_index(frame.index.shift(1, freq=freq), inplace = True)
    frame = frame[3*359:-1]
    
    return frame

In [191]:
lob_techind10 = generate_features(lob_techind10, '10s')
lob_techind10

,mid price,K360,K180,K60,D360,D180,D60,sD360,sD180,sD60,...,ADOSC180,ADOSC60,DISP360,DISP180,DISP60,OSCP180-360,OSCP60-180,RSI360,RSI180,RSI60
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
2016-01-01 02:59:40,430.305,0.000000,0.000000,0.000000,61.295468,34.942988,18.297894,45.643462,55.378159,51.641738,...,1.000000,1.000000,99.891146,99.871226,99.943676,0.000199,-0.000725,49.069940,47.401247,41.388401
2016-01-01 02:59:50,430.305,0.000000,0.000000,0.000000,61.178695,34.452537,17.479712,45.747607,55.188824,50.683026,...,1.000000,1.000000,99.891307,99.872064,99.947062,0.000193,-0.000751,49.069940,46.568110,42.166517
2016-01-01 03:00:00,430.305,0.000000,0.000000,0.000000,61.061923,33.975106,16.510015,45.851132,54.996227,49.701549,...,1.000000,1.000000,99.891468,99.872881,99.950544,0.000186,-0.000778,49.069940,46.641988,41.906475
2016-01-01 03:00:10,430.305,0.000000,0.000000,0.000000,60.945150,33.510697,15.419106,45.954035,54.800515,48.693266,...,1.000000,1.000000,99.891629,99.873680,99.954104,0.000180,-0.000805,49.069940,46.716102,41.696751
2016-01-01 03:00:20,430.305,0.000000,0.000000,0.000000,60.827787,33.146113,14.206984,46.056313,54.602805,47.654137,...,1.000000,1.000000,99.891793,99.874330,99.957742,0.000175,-0.000835,49.050986,47.292225,41.485507
2016-01-01 03:00:30,430.305,0.000000,0.000000,0.000000,60.711014,32.850975,12.903954,46.158406,54.403860,46.581033,...,1.000000,1.000000,99.891954,99.874877,99.961438,0.000171,-0.000867,49.069248,47.701460,41.326067
2016-01-01 03:00:40,430.310,0.000000,0.000000,0.000000,60.613704,32.538475,11.419106,46.260535,54.203146,45.467893,...,1.000000,1.000000,99.892009,99.875450,99.965250,0.000166,-0.000899,49.679608,47.588076,41.004566
2016-01-01 03:00:50,430.305,0.369004,0.369004,0.414938,60.494641,32.219344,9.850264,46.362579,54.000919,44.311953,...,1.000000,1.000000,99.893327,99.877191,99.970263,0.000162,-0.000932,49.065242,47.559653,40.896615
2016-01-01 03:01:00,430.305,0.000000,0.000000,0.000000,60.374554,31.893823,8.183597,46.464510,53.797119,43.110068,...,0.996310,0.995851,99.892328,99.876622,99.973050,0.000157,-0.000965,49.046529,47.505423,40.642202


In [192]:
lob_techind30 = generate_features(lob_techind30, '30s')
lob_techind30

,mid price,K360,K180,K60,D360,D180,D60,sD360,sD180,sD60,...,ADOSC180,ADOSC60,DISP360,DISP180,DISP60,OSCP180-360,OSCP60-180,RSI360,RSI180,RSI60
2016-01-01 08:59:00,433.805,55.862726,48.268156,50.000000,59.314645,53.892303,80.335556,73.952213,66.199069,76.575753,...,0.530726,0.596774,99.980263,99.942713,99.988818,0.000376,-0.000461,48.543689,44.399799,54.328358
2016-01-01 08:59:30,433.355,58.245949,51.061453,70.161290,59.236089,53.620422,79.924971,73.885263,66.188861,77.136373,...,0.517318,0.500000,100.009833,99.974214,100.016908,0.000356,-0.000427,48.692265,44.746693,52.923077
2016-01-01 09:00:00,433.655,49.666349,41.005587,0.000000,59.136247,53.317976,78.431759,73.817585,66.176271,77.649381,...,0.489385,0.291339,99.907097,99.873474,99.913713,0.000337,-0.000403,48.262548,44.271605,47.977685
2016-01-01 09:00:30,433.675,55.386082,47.709497,47.244094,59.052293,53.041470,77.578889,73.749221,66.161379,78.123538,...,0.589944,1.000000,99.977077,99.945333,99.983035,0.000318,-0.000377,48.602740,44.803130,49.457995
2016-01-01 09:01:00,433.755,55.767398,48.156425,50.393701,58.968379,52.767447,76.763429,73.680171,66.144222,78.558569,...,0.522905,0.527559,99.982504,99.952637,99.987800,0.000299,-0.000352,48.602740,44.857073,49.457995
2016-01-01 09:01:30,433.820,57.292660,49.944134,62.992126,58.888194,52.503355,76.199422,73.610442,66.124865,78.957302,...,0.518436,0.496063,100.001720,99.973667,100.005879,0.000281,-0.000322,48.681475,45.071794,51.278600
2016-01-01 09:02:00,433.820,58.531935,51.396648,73.228346,58.811705,52.247333,75.787164,73.540045,66.103348,79.321952,...,0.500559,0.370079,100.017432,99.991158,100.020347,0.000263,-0.000292,48.759698,45.245027,51.797603
2016-01-01 09:02:30,433.840,58.531935,51.396648,73.228346,58.734962,51.991311,75.353790,73.468980,66.079675,79.651814,...,0.486034,0.267717,100.018162,99.993668,100.019943,0.000245,-0.000263,48.754098,45.245027,51.409396
2016-01-01 09:03:00,433.810,58.913251,51.843575,76.377953,58.659532,51.737772,75.037013,73.397243,66.053855,79.953037,...,0.486034,0.267717,100.023488,100.000762,100.023786,0.000227,-0.000230,48.781820,45.298110,52.724796
2016-01-01 09:03:30,433.470,58.341277,51.173184,71.653543,58.586079,51.481047,74.581666,73.324852,66.025799,80.223646,...,0.481564,0.236220,100.017260,99.996363,100.016485,0.000209,-0.000201,48.824494,45.243282,51.377410


In [193]:
lob_techind60 = generate_features(lob_techind60, '60s')
lob_techind60

,mid price,K360,K180,K60,D360,D180,D60,sD360,sD180,sD60,...,ADOSC180,ADOSC60,DISP360,DISP180,DISP60,OSCP180-360,OSCP60-180,RSI360,RSI180,RSI60
2016-01-01 17:58:00,433.890,64.327062,44.792833,6.451613,50.498032,69.982094,46.163657,35.312059,55.487711,42.388190,...,0.366181,0.489247,100.034282,99.894584,99.823031,0.001397,0.000716,49.696396,50.951675,49.002904
2016-01-01 17:59:00,433.630,88.494935,82.194849,96.236559,50.628973,70.186470,47.142260,35.332907,55.633581,42.488038,...,0.552072,0.935484,100.420169,100.276054,100.204328,0.001435,0.000715,50.409679,52.037307,52.074236
2016-01-01 18:00:00,432.145,84.732272,76.371781,82.258065,50.743360,70.347422,47.866137,35.354415,55.781067,42.598903,...,0.178052,0.037634,100.359614,100.211951,100.141045,0.001471,0.000708,50.250170,51.815642,51.516245
2016-01-01 18:01:00,433.675,63.241679,42.272727,2.419355,50.789295,70.346857,47.286765,35.376364,55.929617,42.698084,...,0.239773,0.177419,100.016606,99.866299,99.800677,0.001503,0.000657,49.555518,51.104861,48.851166
2016-01-01 18:02:00,433.675,85.383502,76.318875,84.677419,50.888511,70.529180,48.114155,35.398882,56.080393,42.809791,...,0.595545,0.975806,100.370509,100.215512,100.150473,0.001544,0.000649,50.130581,51.913524,51.501796
2016-01-01 18:03:00,433.465,85.383502,74.686717,84.677419,50.979768,70.689436,48.539308,35.421925,56.233382,42.919319,...,0.253133,0.153226,100.370403,100.211324,100.150126,0.001585,0.000611,50.068485,51.840127,50.151007
2016-01-01 18:04:00,433.545,82.344428,69.423559,73.387097,51.060726,70.706374,48.941719,35.445450,56.387453,43.031099,...,0.253133,0.153226,100.321852,100.160356,100.101091,0.001610,0.000592,49.966844,51.085590,50.235294
2016-01-01 18:05:00,432.270,83.502171,71.428571,77.688172,51.148083,70.732491,49.371623,35.469191,56.542569,43.153034,...,0.305764,0.266129,100.340328,100.176128,100.118987,0.001636,0.000570,50.024863,51.206266,50.252016
2016-01-01 18:06:00,432.190,65.050651,39.473684,9.139785,51.200901,70.582601,48.630645,35.493001,56.697361,43.257478,...,0.285714,0.223118,100.045819,99.880433,99.829046,0.001653,0.000514,49.629995,50.479035,48.112294
2016-01-01 18:07:00,433.375,63.892909,37.468672,4.838710,51.255278,70.430687,47.808512,35.516962,56.851756,43.343372,...,0.605263,0.908602,100.027876,99.860845,99.815430,0.001670,0.000455,49.634076,50.484726,47.963291


In [194]:
lob_techind300 = generate_features(lob_techind300, '300s')
lob_techind300

,mid price,K360,K180,K60,D360,D180,D60,sD360,sD180,sD60,...,ADOSC180,ADOSC60,DISP360,DISP180,DISP60,OSCP180-360,OSCP60-180,RSI360,RSI180,RSI60
2016-01-04 17:50:00,433.490,90.944499,78.397213,77.372263,56.743089,85.371437,59.399938,39.243672,70.720565,56.203226,...,0.305459,0.319951,100.909331,100.495164,100.319711,0.004104,0.001746,51.694725,54.633716,53.820598
2016-01-04 17:55:00,434.060,87.633885,70.499419,69.099757,56.926382,85.238712,58.979721,39.273027,70.971626,56.176565,...,0.216028,0.226277,100.828280,100.413470,100.237052,0.004114,0.001757,51.356694,52.915489,53.114754
2016-01-04 18:00:00,432.875,93.184031,83.739837,82.968370,57.125644,85.148378,59.181642,39.303316,71.219792,56.154397,...,0.295006,0.309002,100.958511,100.542639,100.360694,0.004119,0.001810,51.609154,52.848875,56.261128
2016-01-04 18:05:00,433.020,81.645570,56.213705,54.136253,57.293957,84.905876,59.376962,39.334459,71.464207,56.144608,...,0.162602,0.170316,100.681307,100.266812,100.080824,0.004117,0.001855,51.077195,51.298377,54.373387
2016-01-04 18:10:00,432.780,83.057449,59.581882,57.664234,57.476399,84.681331,59.488902,39.366519,71.704887,56.148471,...,0.437863,0.458637,100.713233,100.298882,100.108600,0.004114,0.001897,51.223614,51.461988,54.281609
2016-01-04 18:15:00,432.795,80.720545,54.006969,51.824818,57.649867,84.425814,59.373186,39.399471,71.941690,56.161859,...,0.404181,0.423358,100.655800,100.242085,100.048933,0.004110,0.001927,51.095600,51.156463,53.104435
2016-01-04 18:20:00,433.065,80.866602,54.355401,52.189781,57.822637,84.172610,59.871767,39.433307,72.174644,56.204406,...,0.459930,0.481752,100.657679,100.244328,100.045192,0.004107,0.001987,51.093439,51.180908,55.592105
2016-01-04 18:25:00,433.010,83.495618,60.627178,58.759124,57.989470,83.954250,60.365304,39.467970,72.404108,56.272545,...,0.456446,0.478102,100.718845,100.305284,100.099912,0.004106,0.002047,51.106400,51.504729,55.921638
2016-01-04 18:30:00,433.590,82.960078,59.349593,57.420925,58.166952,83.728414,60.824689,39.503439,72.629886,56.359491,...,0.393728,0.412409,100.704316,100.291203,100.079777,0.004102,0.002108,51.180998,51.279842,55.700326
2016-01-04 18:35:00,433.760,88.607595,72.822300,71.532847,58.343847,83.584112,61.574565,39.539664,72.852672,56.479341,...,0.406504,0.425791,100.837281,100.423330,100.203891,0.004105,0.002185,51.303772,52.079280,57.401552


In [195]:
lob_techind600 = generate_features(lob_techind600, '600s')
lob_techind600

,mid price,K360,K180,K60,D360,D180,D60,sD360,sD180,sD60,...,ADOSC180,ADOSC60,DISP360,DISP180,DISP60,OSCP180-360,OSCP60-180,RSI360,RSI180,RSI60
2016-01-08 11:40:00,454.460,76.750204,63.195343,36.960179,71.388015,87.945768,28.808812,67.167271,79.685439,58.004236,...,0.341095,0.560292,102.774937,100.256789,99.810040,0.024502,0.004456,55.762445,55.634688,40.847185
2016-01-08 11:50:00,454.430,75.701444,61.376921,32.641615,71.401374,87.810912,29.139863,67.193589,79.867975,57.112791,...,0.369561,0.630398,102.673496,100.154426,99.723113,0.024535,0.004306,55.606402,55.424618,50.895243
2016-01-08 12:00:00,454.740,75.619722,58.245860,32.305104,71.409551,87.655318,28.858212,67.219561,80.048464,56.229162,...,0.416142,0.673584,102.652416,100.130474,99.728803,0.024568,0.004011,55.555834,55.379759,44.969353
2016-01-08 12:10:00,455.250,76.464179,58.935361,35.782389,71.419552,87.445131,28.528406,67.245185,80.226287,55.350947,...,0.425380,0.676949,102.707949,100.183089,99.810544,0.024583,0.003719,55.620252,54.916804,44.320073
2016-01-08 12:20:00,455.345,77.853446,51.024096,41.503085,71.451018,87.177090,28.293946,67.270556,80.400674,54.479586,...,0.520482,0.642176,102.807867,100.279601,99.934347,0.024592,0.003443,55.921267,54.956615,45.166096
2016-01-08 12:30:00,454.800,78.112231,51.596386,42.568704,71.494157,86.908181,28.048445,67.295765,80.571364,53.612955,...,0.489759,0.584969,102.813720,100.291635,99.967416,0.024531,0.003233,56.059923,52.908120,45.010457
2016-01-08 12:40:00,455.425,76.627622,48.313253,43.449198,71.537867,86.638531,27.547799,67.320833,80.738352,52.742036,...,0.484036,0.492647,102.675320,100.162557,99.868467,0.024473,0.002936,55.957590,52.963395,41.212967
2016-01-08 12:50:00,455.910,78.330155,52.078313,51.804813,71.585785,86.372300,27.633586,67.345788,80.901570,51.880865,...,0.516867,0.565508,102.800670,100.291534,100.013359,0.024408,0.002774,56.090447,52.845921,46.506185
2016-01-08 13:00:00,455.375,79.651321,55.000000,58.288770,71.641806,86.122300,27.666760,67.370644,81.061015,51.025148,...,0.479217,0.481952,102.893962,100.389091,100.129634,0.024344,0.002585,56.243933,53.021912,45.457644
2016-01-08 13:10:00,455.535,78.193953,51.777108,51.794177,71.665741,85.857338,27.522721,67.395239,81.217214,50.172260,...,0.450000,0.409614,102.758096,100.262566,100.025517,0.024285,0.002364,55.858936,52.843772,43.787183


In [196]:
lob_techind10.to_csv(path_or_buf='../btc-data/BTC_LOB_techind_10s.csv')
lob_techind30.to_csv(path_or_buf='../btc-data/BTC_LOB_techind_30s.csv')
lob_techind60.to_csv(path_or_buf='../btc-data/BTC_LOB_techind_60s.csv')
lob_techind300.to_csv(path_or_buf='../btc-data/BTC_LOB_techind_300s.csv')
lob_techind600.to_csv(path_or_buf='../btc-data/BTC_LOB_techind_600s.csv')